# Função para receber o gênero de uma música do dataset de acordo com seu ID

In [97]:
import pandas as pd

def check_gender_by_id(filename, id) -> None | str:
    df = pd.read_excel(filename)

    genero = df.loc[df['id'] == id, 'genero']
    return genero.iloc[0]

In [98]:
# Exemplo de uso
arquivo = '/content/generos.xlsx'
id_musica = 17  # Coloque o ID desejado
genero = check_gender_by_id(arquivo, id_musica)
print(f'O gênero da música com ID {id_musica} é: {genero}')

O gênero da música com ID 17 é: Pop


In [99]:
def dividir_musicas_em_listas(arquivo_xlsx):
    # Carrega o arquivo Excel para um DataFrame
    df = pd.read_excel(arquivo_xlsx)

    # Dicionário para armazenar as músicas de cada gênero
    generos = {}

    # Itera por todas as músicas no arquivo
    for id_musica in df['id']:
        # Usa a função check_gender_by_id para obter o gênero da música
        genero = check_gender_by_id(arquivo_xlsx, id_musica)

        # Se o gênero for válido (não None e dentro da lista de gêneros válidos), adiciona o ID
        if genero not in generos:
            generos[genero] = []
        generos[genero].append(id_musica)

    # Listas para armazenar as músicas de treino, validação e teste
    musicas_treino = []
    musicas_validacao = []
    musicas_teste = []

    # Agora, vamos dividir as músicas por gênero em treino, validação e teste de forma aleatória
    for genero, musicas in generos.items():
        # Embaralha as músicas para aleatoriedade
        random.shuffle(musicas)

        # Verifica se há pelo menos 30 músicas para dividir

        if len(musicas)<10:
          print(f"Erro: genero {genero} com poucas músicas")
          return
        else:
          total = len(musicas)

          idx_70 = int(0.7 * total) #limites para dividir com qualquer tamanho
          idx_90 = int(0.9 * total)

          treino = musicas[:idx_70]
          validacao = musicas[idx_70:idx_90]
          teste = musicas[idx_90:]

          # Adiciona as listas para cada categoria
          musicas_treino.extend(treino)
          musicas_validacao.extend(validacao)
          musicas_teste.extend(teste)


    # Retorna as 3 listas
    return musicas_treino, musicas_validacao, musicas_teste

In [100]:
import random
treino,validacao,teste = dividir_musicas_em_listas('/content/generos.xlsx')

Exception ignored in: <function _xla_gc_callback at 0x7fe07dc9db40>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/jax/_src/lib/__init__.py", line 96, in _xla_gc_callback
    def _xla_gc_callback(*args):
KeyboardInterrupt: 


In [101]:
for x in treino:
  print(check_gender_by_id('/content/generos.xlsx',x))

Rock
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Pop
Pop
Pop
Pop
Pop
Pop
Pop
Pop
Pop
Pop
Pop
Pop
Pop
Pop
Pop
Pop
Pop
Pop
Pop
Pop
Pop
Eletronical
Eletronical
Eletronical
Eletronical
Eletronical
Eletronical
Eletronical
Eletronical
Eletronical
Eletronical
Eletronical
Eletronical
Eletronical
Eletronical
Eletronical
Eletronical
Eletronical
Eletronical
Eletronical
Eletronical
Eletronical
Classical
Classical
Classical
Classical
Classical
Classical
Classical
Classical
Classical
Classical
Classical
Classical
Classical
Classical
Classical
Classical
Classical
Classical
Classical
Classical
Classical
Jazz
Jazz
Jazz
Jazz
Jazz
Jazz
Jazz
Jazz
Jazz
Jazz
Jazz
Jazz
Jazz
Jazz
Jazz
Jazz
Jazz
Jazz
Jazz
Jazz
Jazz
Country
Country
Country
Country
Country
Country
Country
Country
Country
Country
Country
Country
Country
Country
Country
Country
Country
Country
Country
Country
Country


In [102]:
# Exemplo: Gêneros associados aos IDs
id_to_genre = {
    0: "Rock",
    1: "Pop",
    2: "Jazz",
    3: "Eletronical",
    4: "Classical",
    5: "Country"
    # Adicione todos os IDs e seus gêneros
}

# Converta os gêneros para índices numéricos para o treinamento
genre_to_index = {genre: idx for idx, genre in enumerate(set(id_to_genre.values()))}

# Exemplo: rock -> 0, pop -> 1, jazz -> 2
id_to_label = {id: genre_to_index[genre] for id, genre in id_to_genre.items()}

print(id_to_genre)
print(genre_to_index)
print(id_to_label)


{0: 'Rock', 1: 'Pop', 2: 'Jazz', 3: 'Eletronical', 4: 'Classical', 5: 'Country'}
{'Jazz': 0, 'Classical': 1, 'Rock': 2, 'Eletronical': 3, 'Country': 4, 'Pop': 5}
{0: 2, 1: 5, 2: 0, 3: 3, 4: 1, 5: 4}


In [103]:
import tensorflow as tf
import os


base_dir = "/content/espectrogramas"

def id_to_filepath(id):
  return os.path.join(base_dir,f"{id}.png")

In [104]:
df = pd.read_excel('/content/generos.xlsx')

# Cria o mapeamento de gêneros para índices
genre_to_index = {genre: idx for idx, genre in enumerate(set(df['genero']))}

# Mapeia IDs das músicas aos índices das classes
id_to_label = {row['id']: genre_to_index[row['genero']] for _, row in df.iterrows()}

print("id_label:", id_to_label)
print(genre_to_index)

id_label: {1: 2, 2: 2, 3: 2, 4: 2, 5: 2, 6: 2, 7: 2, 8: 2, 9: 2, 10: 2, 11: 5, 12: 5, 13: 5, 14: 5, 15: 5, 16: 5, 17: 5, 18: 5, 19: 5, 20: 5, 21: 3, 22: 3, 23: 3, 24: 3, 25: 3, 26: 3, 27: 3, 28: 3, 29: 3, 30: 3, 31: 1, 32: 1, 33: 1, 34: 1, 35: 1, 36: 1, 37: 1, 38: 1, 39: 1, 40: 1, 41: 0, 42: 0, 43: 0, 44: 0, 45: 0, 46: 0, 47: 0, 48: 0, 49: 0, 50: 0, 51: 4, 52: 4, 53: 4, 54: 4, 55: 4, 56: 4, 57: 4, 58: 4, 59: 4, 60: 4, 61: 2, 62: 2, 63: 2, 64: 2, 65: 2, 66: 2, 67: 2, 68: 2, 69: 2, 70: 2, 71: 2, 72: 2, 73: 2, 74: 2, 75: 2, 76: 2, 77: 2, 78: 2, 79: 2, 80: 2, 81: 5, 82: 5, 83: 5, 84: 5, 85: 5, 86: 5, 87: 5, 88: 5, 89: 5, 90: 5, 91: 5, 92: 5, 93: 5, 94: 5, 95: 5, 96: 5, 97: 5, 98: 5, 99: 5, 100: 5, 101: 3, 102: 3, 103: 3, 104: 3, 105: 3, 106: 3, 107: 3, 108: 3, 109: 3, 110: 3, 111: 3, 112: 3, 113: 3, 114: 3, 115: 3, 116: 3, 117: 3, 118: 3, 119: 3, 120: 3, 121: 1, 122: 1, 123: 1, 124: 1, 125: 1, 126: 1, 127: 1, 128: 1, 129: 1, 130: 1, 131: 1, 132: 1, 133: 1, 134: 1, 135: 1, 136: 1, 137: 1, 1

In [105]:
def create_tf_dataset_with_labels(id_list, id_to_label):
    filepaths = [id_to_filepath(id) for id in id_list]
    labels = [id_to_label[id] for id in id_list]

    # Cria um dataset contendo pares (caminho, rótulo)
    dataset = tf.data.Dataset.from_tensor_slices((filepaths, labels))

    # Carrega a imagem e retorna (imagem, rótulo)
    def load_image_and_label(filepath, label):
        image = tf.io.read_file(filepath)
        image = tf.image.decode_png(image, channels=3)
        image = tf.image.resize(image, [224, 224])  # Tamanho esperado pela CNN
        return image, label

    dataset = dataset.map(load_image_and_label, num_parallel_calls=tf.data.AUTOTUNE)
    return dataset


In [ ]:
def normalizar_imagem(imagem):
    return tf.cast(imagem, tf.float32) / 255.0  # Normaliza para o intervalo [0, 1]

train_dataset = create_tf_dataset_with_labels(treino, id_to_label)
val_dataset = create_tf_dataset_with_labels(validacao, id_to_label)
test_dataset = create_tf_dataset_with_labels(teste, id_to_label)

train_dataset = train_dataset.map(lambda imagem, label: (normalizar_imagem(imagem), label))
val_dataset = val_dataset.map(lambda imagem, label: (normalizar_imagem(imagem), label))
test_dataset = test_dataset.map(lambda imagem, label: (normalizar_imagem(imagem), label))


# Agrupar em lotes e otimizar
batch_size = 32
train_dataset = train_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
val_dataset = val_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)


In [ ]:
import cv2 as cv

img = cv.imread("/content/espectrogramas/125.png")


input_shape = img.shape
print(input_shape)
num_classes = len(set(id_to_label.values()))
print(f"Quantidade de classes: {num_classes}")

print(num_classes)

In [ ]:
import os
from PIL import Image

def verificar_tamanhos_canais_pasta(pasta):
    tamanhos = {}
    canais = {}
    arquivos_verificados = 0

    for root, _, files in os.walk(pasta):
        for file in files:
            if file.endswith((".png", ".jpg", ".jpeg")):  # Ajuste as extensões conforme necessário
                caminho_completo = os.path.join(root, file)
                try:
                    with Image.open(caminho_completo) as img:
                        tamanho = img.size  # (largura, altura)
                        modo = img.mode  # Ex.: "L", "RGB", "RGBA"
                        canais_count = {'L': 1, 'RGB': 3, 'RGBA': 4}.get(modo, "Desconhecido")

                        tamanhos[caminho_completo] = tamanho
                        canais[caminho_completo] = canais_count
                        arquivos_verificados += 1
                except Exception as e:
                    print(f"Erro ao processar {caminho_completo}: {e}")

    if arquivos_verificados == 0:
        print("Nenhum arquivo de imagem foi encontrado na pasta especificada.")
        return

    # Identificar se todos têm o mesmo tamanho e canais
    tamanhos_unicos = set(tamanhos.values())
    canais_unicos = set(canais.values())

    print(f"\nTotal de imagens verificadas: {arquivos_verificados}")
    if len(tamanhos_unicos) == 1:
        print(f"Todos os espectrogramas têm o mesmo tamanho: {tamanhos_unicos.pop()}")
    else:
        print("Os tamanhos dos espectrogramas variam. Detalhes:")
        for caminho, tamanho in tamanhos.items():
            print(f"{caminho}: {tamanho}")

    if len(canais_unicos) == 1:
        print(f"Todas as imagens têm o mesmo número de canais: {canais_unicos.pop()}")
    else:
        print("As imagens têm diferentes números de canais. Detalhes:")
        for caminho, num_canais in canais.items():
            print(f"{caminho}: {num_canais} canais")

# Substitua pelo caminho da sua pasta de espectrogramas
caminho_pasta = "/content/espectrogramas"
verificar_tamanhos_canais_pasta(caminho_pasta)

In [ ]:
cnn = tf.keras.models.Sequential()
cnn.add(tf.keras.layers.Conv2D(filters=32,kernel_size=3, activation='relu', input_shape=(2325,924,3)))
cnn.add(tf.keras.layers.MaxPool2D(pool_size = 2))
cnn.add(tf.keras.layers.Conv2D(filters=64,kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size = 2))
cnn.add(tf.keras.layers.Conv2D(filters=128,kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size = 2))
cnn.add(tf.keras.layers.GlobalAveragePooling2D())
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))
cnn.add(tf.keras.layers.Dropout(0.5))
cnn.add(tf.keras.layers.Dense(units=num_classes, activation='softmax'))
cnn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

cnn.summary()

In [ ]:
history = cnn.fit(train_dataset, epochs=30, validation_data=val_dataset)